In [21]:
import pandas as pd

In [22]:
prefix_path = 'D:/jeju_bus_data_no_leakage/bis/'
infix_path = 'BIS_' + '190531'
postfix_path = '.csv'

path = prefix_path + infix_path + postfix_path

In [23]:
df = pd.read_csv(path, 
                 usecols = ['EVENT_DATE', 'EVENT_CD', 'RUN_DIST',
                            'ROUTE_ID', 'ROUTE_RUN_CNT', 'VH_ID', 'BUS_RUN_CNT', 'STATION_ORD'])

In [31]:
# df = df[(df['EVENT_CD'] == 18) | (df['EVENT_CD'] == 17)]
df = df.query('EVENT_CD in (17, 18)')

In [27]:
grouped = df.groupby(by = ['ROUTE_ID', 'ROUTE_RUN_CNT', 'VH_ID', 'BUS_RUN_CNT'])

In [12]:
instances = []

for name, group in grouped:
#     print(name)
    sorted_group = group.sort_values(by = ['EVENT_DATE', 'EVENT_CD'], ascending = [True, True])
    
    # 버스 출발/도착 이벤트 로그가 중복되어 저장된 경우 -> 해당 로그 튜플 삭제
    # 예) EVENT_CD | STATION_ORD (2019.05.31 BIS, 노선ID 405130001, 6회차)
    #     ---------+------------
    #     18       | 1             -> 삭제 대상
    #     17       | 1             -> 삭제 대상
    #     18       | 1             -> 삭제 대상
    #     17       | 1             -> 삭제 대상
    #     18       | 1
    for station_ord, gr in sorted_group.groupby(by = 'STATION_ORD'):
        if len(gr) > 2:
            #print(name, '...', gr)
            sorted_group = sorted_group.drop(gr.index[:-1])
    
    # 첫 행의 EVENT_CD가 18(출발)이 아닌 경우 -> 해당 로그 튜플 삭제
    if len(sorted_group) > 0 and \
            sorted_group.loc[sorted_group.index[0]]['EVENT_CD'] == 17:
        sorted_group = sorted_group.drop(sorted_group.index[0])
    
    # 마지막 행의 EVENT_CD가 17(도착)이 아닌 경우 -> 해당 로그 튜플 삭제
    if len(sorted_group) > 0 and \
            sorted_group.loc[sorted_group.index[-1]]['EVENT_CD'] == 18:
        sorted_group = sorted_group.drop(sorted_group.index[-1])
        
    # 출발/도착 결측 값 보정
    tup = (len(sorted_group[sorted_group['EVENT_CD'] == 18]), len(sorted_group[sorted_group['EVENT_CD'] == 17]))
    if tup[0] != tup[1]:
        cnt, start, end = -1, 0, len(sorted_group) - 1
        for station_ord, gr in sorted_group.groupby(by = 'STATION_ORD'):
            cnt += 1
            if cnt in (start, end): 
                continue
            
            if len(gr) == 1:
                if gr.loc[gr.index[0]]['EVENT_CD'] == 18:
                    pass
                else: # 17
                    pass
                
    start_df = sorted_group.query('EVENT_CD == 18')
    end_df = sorted_group.query('EVENT_CD == 17')

    
#         print(name, end = ' ... ')
#         print(tup)
#         instances.append(sorted_group)

(405130001, 6, 7996023, 7) ... (23, 22)
(405130002, 9, 7996007, 3) ... (21, 20)
(405130002, 18, 7996024, 4) ... (8, 7)
(405131101, 5, 7983683, 6) ... (51, 52)
(405131101, 10, 7983683, 2) ... (46, 47)
(405131101, 22, 7983683, 4) ... (46, 48)
(405131102, 17, 7983682, 6) ... (56, 55)
(405131102, 22, 7983683, 5) ... (56, 55)
(405131201, 2, 7983679, 5) ... (34, 35)
(405131201, 5, 7983676, 2) ... (62, 60)
(405131201, 9, 7983679, 7) ... (63, 61)
(405131201, 14, 7983676, 4) ... (62, 64)
(405131201, 23, 7983679, 4) ... (22, 23)
(405131202, 4, 7983693, 1) ... (63, 61)
(405131202, 6, 7983694, 2) ... (72, 71)
(405131202, 9, 7983676, 3) ... (66, 65)
(405131202, 12, 7983693, 3) ... (55, 57)
(405131202, 16, 7983679, 3) ... (66, 65)
(405131202, 17, 7983676, 5) ... (60, 62)
(405131501, 7, 7983176, 7) ... (63, 64)
(405131601, 7, 7983177, 8) ... (67, 68)
(405131602, 2, 7983153, 1) ... (1, 0)
(405131602, 2, 7983158, 6) ... (41, 40)
(405132002, 10, 7997096, 13) ... (47, 44)
(405132005, 1, 7997010, 1) ... (

(406151001, 0, 7997544, 6) ... (10, 9)
(406151001, 10, 7997544, 2) ... (58, 60)
(406151001, 21, 7997505, 3) ... (76, 75)
(406151001, 23, 7997544, 9) ... (3, 4)
(406151001, 41, 7997519, 6) ... (30, 29)
(406151001, 42, 7997569, 5) ... (15, 14)
(406151001, 46, 7997506, 6) ... (7, 6)
(406151001, 57, 7997544, 7) ... (10, 11)
(406151002, 1, 7997512, 1) ... (74, 73)
(406151002, 19, 7997569, 11) ... (0, 1)
(406151002, 25, 7997569, 4) ... (53, 51)
(406151008, 6, 7997544, 10) ... (0, 1)
(406152001, 0, 7997554, 2) ... (64, 63)
(406152001, 3, 7997554, 9) ... (67, 66)
(406152002, 28, 7997554, 8) ... (23, 21)
(406152003, 1, 7997554, 11) ... (42, 41)
(406153101, 11, 7997564, 5) ... (63, 62)
(406153102, 7, 7997539, 11) ... (59, 58)
(406153104, 2, 7997539, 1) ... (62, 63)
(406153201, 4, 7997567, 8) ... (47, 46)
(406153203, 1, 7997568, 1) ... (69, 68)
(406261101, 6, 7983206, 7) ... (37, 36)
(406261507, 1, 7983208, 1) ... (27, 28)
(406262101, 4, 7997517, 3) ... (78, 77)
(406262201, 1, 7996615, 1) ... (47

In [41]:
start_df = sorted_group.query('EVENT_CD == 18')
end_df = sorted_group.query('EVENT_CD == 17')

In [56]:
start_df[['EVENT_DATE', 'RUN_DIST', 'STATION_ORD']]

,EVENT_DATE,RUN_DIST,STATION_ORD
1886985,19/05/31 23:00:21,0,1
1899498,19/05/31 23:00:50,318,2
1915054,19/05/31 23:01:41,1105,3
1930422,19/05/31 23:02:25,1508,4
1915072,19/05/31 23:04:13,1852,5
1933581,19/05/31 23:05:23,2435,6
1902777,19/05/31 23:06:29,2822,7
1921326,19/05/31 23:07:43,3119,8
1899668,19/05/31 23:09:42,3370,9
1909018,19/05/31 23:10:25,3558,10


In [45]:
end_df[['EVENT_DATE', 'RUN_DIST', 'STATION_ORD']]

,EVENT_DATE,RUN_DIST,STATION_ORD
1896369,19/05/31 23:00:43,318,2
1915048,19/05/31 23:01:40,1105,3
1927371,19/05/31 23:02:21,1508,4
1911986,19/05/31 23:04:04,1852,5
1930497,19/05/31 23:05:12,2435,6
1893370,19/05/31 23:05:56,2822,7
1921317,19/05/31 23:07:42,3119,8
1893393,19/05/31 23:09:13,3370,9
1902842,19/05/31 23:10:06,3558,10
1918276,19/05/31 23:11:12,4059,11


In [52]:
x